# Парсер лотов с сайта torgi.gov.ru 

Парсер настроен только на категорию лотов "Земельные участки"

### Импортируем бибилиотеки 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html
from selenium import webdriver
import time
import datetime 

In [2]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

### Передаём URL ссылку и читаем страницу с помощью BeautifulSoup

In [3]:
url = 'https://torgi.gov.ru/new/api/public/notices/rss?subjRF=50&biddEndFrom=&biddEndTo=&pubFrom=2022-09-13&pubTo=2022-09-13&aucStartFrom=&aucStartTo=&catCode=2&text='

req = requests.get(url, verify=False)

soup = BeautifulSoup(req.text, features='xml')

In [4]:
#soup

### В этой строке я ищу описание лотов в коде страницы и добавляю описание лотов в список

In [5]:
num = soup.find_all('item')
description_list = []
for index in num:
    col = index.find(name='description')
    sel = html.unescape(col.text)
    description_list.append(sel)
#print(description_list)

### На этом этапе я выделяю ссылку на каждый лот из списка с описаниями, чтобы в дальнейшем парсить каждый лот по ссылке

In [6]:
links_all = []
for i in description_list:
    b = BeautifulSoup(i, features='html')
    link = b.find('a')
    link_final = link.get('href')
    links_all.append(link_final)

# -------------------------------------------------------------------------------------------------------------

### Этот код нужно использовать если сайт перестал отдавать полный html страницы при поиске лотов. Ищем на сайте не лоты, а извещения, получаем код страницы и ссылки на извещения, затем переделываем ссылки из ссылок на извещения в ссылки на лоты.

In [7]:
for index in range(0,len(links_all)):
    links_all[index] = 'https://torgi.gov.ru/new/public/lots/lot/' + links_all[index][-20:] + '_1/(lotInfo:info)'
#links_all    

# -------------------------------------------------------------------------------------------------------------

### Этот код для заполнения ссылками списка вручную если не получилось достать код страницы 

In [8]:
#links_all = [
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000002786_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000002787_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000002785_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000002778_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000002777_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000001977_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000001552_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000001992_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000001962_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000001951_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000001946_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000001945_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000000899_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000000898_1/(lotInfo:info)',
#    'https://torgi.gov.ru/new/public/lots/lot/21000004710000000895_1/(lotInfo:info)'
#]

In [9]:
#!pip install webdriver_manager

### С помощью этого кода открывается окно браузера (нужно только скачать вебдрайвер для конкретного браузера и указать путь к нему)

In [10]:
driver = webdriver.Edge('C:\\Users\\KonovalovAlE\\Desktop\\jupyter notebook\\Вебдрайвер edge\\msedgedriver.exe')

C:\Users\KonovalovAlE\AppData\Local\Temp\ipykernel_16916\171963719.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('C:\\Users\\KonovalovAlE\\Desktop\\jupyter notebook\\Вебдрайвер edge\\msedgedriver.exe')


### Это функция которая достаёт из ссылки, нужные мне данные и где нужно изменяет их, после чего добавляет каждый элемент в список и возвращает список выгруженных данных на выходе

In [11]:
def data_parse(link):
    driver.get(link)
    time.sleep(2)
    main_page = driver.page_source
    soup = BeautifulSoup(main_page, features='xml')
    
    # Ищу местоположение
    pl = soup.find_all(attrs={"class":{"attr_value"}})
    place = pl[18].text
    
    # Площадь участка 
    sq = soup.find_all(attrs={"class":{"lotAttributeDetail no-padding-bottom"}})
    square = sq[0].text.split('м')[0].strip()
    square
    
    # Описание лота
    head = soup.find_all(attrs={"class":{"header_title"}})
    header = head[0].text
    
    # Срок аренды
    rent = soup.find_all(attrs={"class":{"attr_value"}})
    try:
        rent_time = rent[23].text.strip()
    except:
        rent_time = 'не нашёл'
    if len(rent_time) == 0:
        try:
            rent_time = rent[24].text.strip()
        except:
            if len(rent_time) == 0:
                rent_time = rent[17].text.strip() + ' месяца'
                if rent_time == 'в соответствии с Извещением месяца':
                    rent_time = rent[16].text.strip() + ' месяца'
    if rent_time == 'Не разграниченная собственность':
        rent_time = ''
    
    # Начальная цена
    start = soup.find_all(attrs={"class":{"prices__row__price-cell lotPrice"}}) 
    start_price = start[0].text.split('₽')[0]
    
    # Номер извещения
    
    num = soup.find_all(attrs={"class":{"lotAttributeValue"}})
    nums = num[0].text.split(',')
    number = nums[0].split('№')[1]
    
    # Номер лота
    
    lot = nums[1].split('№')
    lot_n = lot[0]+lot[1]
    lot_num = lot_n[1:]
    
    # Ссылка на лот, добавляю в итоговые данные по порядку
    
    #link которая передаётся в функцию

    # Дата окончания приема заявок
    
    date_cl = soup.find_all(attrs={"class":{"lotAttributeValue"}})
    date_close = date_cl[5].text[1:11]
    
    
    # Дата торгов
    
    date_st = soup.find_all(attrs={"class":{"lotAttributeValue"}})
    date_start = date_st[6].text[1:11]
    
    # Назначение 
    
    Appointment = soup.find_all(attrs={"class":{"lotAttributeDetail no-padding-bottom"}})[-1].text.strip()
    if "50:" in Appointment:
        try:
            Appointment = soup.find_all(attrs={"class":{"lotAttributeDetail no-padding-bottom"}})[-2].text.strip()
        except:
            Appointment = soup.find_all(attrs={"class":{"lotAttributeDetail no-padding-bottom"}})[-3].text.strip()
    
    # Статус лота
    try:
        status = soup.find_all(attrs={"class":{"header_status prague"}})
        if len(list(status)) == 0:
            status = soup.find_all(attrs={"class":{"header_status amsterdam"}})[0].text.strip()
        else:
            status = soup.find_all(attrs={"class":{"header_status prague"}})[0].text.strip()
    except:
        status
        
    # Количество версий
        #Достаю ссылку на извещение 
    linked = soup.find_all(attrs={"class":{"button text"}})
    link_to = 'https://torgi.gov.ru' + linked[7].get('href')
    link_to
        #Загружаю ссылку через драйвер
    driver.get(link_to)
    time.sleep(2)
    main_page = driver.page_source
    soup = BeautifulSoup(main_page, features='xml')
    
    # Дата публикации 
    lot = soup.find_all(attrs={"class":{"attr_value"}})
    dt_publ = lot[1].text[1:11]
    
    # Организатор торгов
    for i in lot:
        if 'КОМИТЕТ' in i.text:
            auction_org = i.text
        else:
            auction_org = lot[8].text
        
    # Получаю количество версий 
    versions = driver.find_element("class name", 'aside-btn')
    versions.click()
    time.sleep(2)
    action_time = driver.find_element("class name", "version-link").get_attribute('innerHTML').strip()
    
    ###############################################
    
    list_of_data = [auction_org,place,square,header,rent_time,start_price,number,lot_num,link, dt_publ,date_close,date_start,Appointment,status, action_time]
    
    return list_of_data

### Код для проверки работоспособности функции на первой ссылке из списка (первый лот)

In [12]:
#data_parse(links_all[0])

### Создаю датафрейм, названия колонок соответствуют порядку выгружаемых данных с помощью функции чтобы ничего не перепуталось в таблице

In [13]:
index = ['Организатор торгов','местоположение','Площадь ЗУ (кв.м)','Описание объекта','Срок передачи прав(лет)','Начальнаяцена,загод','Номер извещения','Номер лота','Ссылка','Дата публикации','Дата окончания приема заявок','Дата торгов/ подведения итогов/отмены','Назначение','Статус лота','Количество версий']
columns = list(range(0,len(links_all)))
data = pd.DataFrame(index=index, columns=columns)
#data

### заполняю таблицу

Код для заполнения безопасным способом, если на одном из лотов функция не сработает строка соответствующая этому лоту останется пустой в таблице

In [14]:
#for column in data.columns:
#    try:
#        data[column] = data_parse(links_all[column])
#    except:
#        data[column] = data[column]

Стандартный способ заполнения таблицы

In [15]:
for column in data.columns:
        data[column] = data_parse(links_all[column])

In [16]:
data = data.T

### Функция для замены ИЖС и ЛПХ в назначении

In [17]:
def izs_lpx(data):
    if data == 'Для индивидуального жилищного строительства':
        return 'ИЖС'
    elif data == 'Для ведения личного подсобного хозяйства (приусадебный земельный участок)':
        return 'ЛПХ'
    else:
        return data

### Меняю данные в столбце "Назначение"

In [18]:
data['Назначение'] = data['Назначение'].apply(izs_lpx)

Меняю тип данных столбца `Номер извещения` в строковый на всякий случай

In [19]:
data['Номер извещения'] = data['Номер извещения'].astype('str')

### Особенности и предварительный просмотр

Когда правим выгрузку в Excel нужно для каждого столбца указать тип данных, чтобы номер извещения считывался как тект и не переводился в экспоненциальное число автоматически!!!

Сортирую таблицу по количеству версий 

In [20]:
data = data.sort_values(by='Количество версий').reset_index(drop=True)

Предварительный просмотр таблицы

In [21]:
data

,Организатор торгов,местоположение,Площадь ЗУ (кв.м),Описание объекта,Срок передачи прав(лет),"Начальнаяцена,загод",Номер извещения,Номер лота,Ссылка,Дата публикации,Дата окончания приема заявок,Дата торгов/ подведения итогов/отмены,Назначение,Статус лота,Количество версий
0,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Московская область, Талдомский г.о, с. Квашенки",627,Аренда земельного участка 627 кв.м. для ЛПХ в ...,20 лет,"8296,96",21000004710000002889,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,25.10.2022,28.10.2022,ЛПХ,Опубликован,Версия 1
1,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Российская Федерация, Московская область, Горо...",3000,Аренда земельного участка 3 000 кв.м для ЛПХ в...,13 лет 2 месяцев,"144795,00",21000004710000002888,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,26.10.2022,31.10.2022,ЛПХ,Опубликован,Версия 1
2,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Московская область, рп Серебряные Пруды, мкр Т...",980,Аренда земельного участка 980 кв.м. для ЛПХ в ...,20 лет,"26435,01",21000004710000002887,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,31.10.2022,03.11.2022,ЛПХ,Опубликован,Версия 1
3,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Российская Федерация, Московская область, Горо...",732,Аренда земельного участка 732 кв.м. для ИЖС в ...,20 лет,"38247,73",21000004710000002885,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,26.10.2022,31.10.2022,ИЖС,Опубликован,Версия 1
4,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Российская Федерация, Московская область, Талд...",1259,Аренда земельного участка 1 259 кв.м. для ИЖС ...,20 лет,"30098,53",21000004710000002884,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,25.10.2022,28.10.2022,ИЖС,Опубликован,Версия 1
5,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Российская Федерация, Московская область, Горо...",1390,Аренда земельного участка 1 390 кв.м. для ИЖС ...,20 лет,"72425,95",21000004710000002883,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,26.10.2022,31.10.2022,ИЖС,Опубликован,Версия 1
6,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Московская область, Дмитровский городской окру...",1976,Аренда земельного участка 1 976 кв.м. для ИЖС ...,20 лет,"841756,00",21000004710000002882,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,26.10.2022,31.10.2022,ИЖС,Опубликован,Версия 1
7,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,Местоположение установлено относительно ориент...,1124,Аренда земельного участка 1 124 кв.м. для ИЖС ...,13 лет 2 месяцев,"15596,06",21000004710000002881,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,25.10.2022,28.10.2022,ИЖС,Опубликован,Версия 1
8,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Российская Федерация, Московская область, Горо...",2650,Аренда земельного участка 2 650 кв.м. для ЛПХ ...,20 лет,"134339,10",21000004710000002880,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,26.10.2022,31.10.2022,ЛПХ,Опубликован,Версия 1
9,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Московская область, Дмитровский городской окру...",702,Аренда земельного участка 702 кв.м. для ЛПХ в ...,20 лет,"592846,02",21000004710000002879,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,26.10.2022,31.10.2022,ЛПХ,Опубликован,Версия 1


### Сохранение

Сохраняю таблицу в кодировке "utf-8", считываем её в той же кодировке в excel, здесь указывается название файла

In [24]:
#data.to_csv('parse_15_09_2022.csv', encoding='utf-8')

Заходим на Home Page и скачиваем получившийся файл в формате csv 

Далее кладём его в любую папку и открываем, внутри непонятные строки, удаляем всё, нажимаем на вкладку данные и добавить из текста, выбираем этот файл в папке из которой его открыли, импортируем с разделителем "запятая" и для каждой колонки проставляем тип данных тектовый, получаем в результате готовую таблицу, сохраняем её как лист или книгу excel 